# Content-Based Filtering

### 1. 개요
- **사용자가 선호하는 아이템의 콘텐츠 정보**와 유사한 아이템을 추천하는 방식
    - 콘텐츠 정보 예시 : 장르, 키워드, 설명 등

### 2. 장단점
- 장점
    - 설명 가능성 높음 (Explainability)
    - Cold Start Problem에 상대적으로 덜 민감 (CF에 비해)
        - 새로운 유저인 경우, 회원가입 시 선호 장르 등에 대한 정보 받을 수 있음
        - 새로운 컨텐츠의 경우, 다른 유저의 평점이 없어도 메타 데이터로 추천 가능
- 단점
    - 기존 선호도와 유사한 아이템만 추천 (새로운 취향 발견 어려움, Filter Bubble)
    - 아이템의 정보가 충분해야 가능

### 3. 구현
1. Feature Extraction
    - 아이템의 주요 특징을 추출
    - 예시
        - 영화 : 장르, 감독, 출연 배우, 줄거리
        - 상품 : 카테고리, 가격, 브랜드
2. Embedding
    - 아이템의 특징을 벡터로 변환
        - 아이템 피쳐를 만드는 방법
            - concat([장르, 감독, 줄거리, ...]) : 단순 결합 후 임베딩이 구현 쉬움
            - embedding(장르), embedding(감독)의 가중 평균 : 개별 임베딩 후 결합하면 각 특성에 대한 가중치 부여도 가능
            - 정답은 없음
    - 주요 방법
        - TF-IDF : 단어 빈도 기반
        - Word2Vec : 단어 의미 학습
        - BERT Embedding : 문맥 반영
            - Transformer 모델, 동음이의어 뜻 차이 반영 가능
3. Similarity Calculation
    - 유사도 계산
    - 주요 방법
        - Cosine Similarity
        - Euclidian Distance
4. Recommendation
    - 사용자가 선호하는 아이템과 유사도가 높은 아이템 추천
    - 예시
        - 센과 치히로 좋아하는 사용자에게 하울의 움직이는 성 추천
        - viva la vida 좋아하는 사용자에게 fix you 추천

### 4. 활용 사례
- Netflix 비슷한 장르 영화 추천
- Spotify 비슷한 장르 음악 추천
- Coupang 동일 브랜드 상품 추천

### 5. 관련 수식
#### 1. 아이템 벡터 표현
각 아이템 $ i $ 는 특성(feature) 벡터 $ \mathbf{x}_i $ 로 표현됨

$$
\mathbf{x}_i = (x_{i1}, x_{i2}, ..., x_{in})
$$

#### 2. 사용자 프로파일 벡터
사용자 $ u $ 는 선호하는 아이템들의 가중합으로 표현됨

$$
\mathbf{p}_u = \frac{\sum_{i \in I_u} w_{u,i} \mathbf{x}_i}{\sum_{i \in I_u} w_{u,i}}
$$

여기서
- $ I_u $ : 사용자 $ u $ 가 평가한 아이템 집합
- $ w_{u,i} $ : 사용자 $ u $ 가 아이템 $ i $ 에 부여한 가중치(예: 평점)
- $ \mathbf{x}_i $ : 아이템 $ i $ 의 특성 벡터

#### 3. 유사도 계산
사용자 프로파일 $ \mathbf{p}_u $ 와 아이템 $ \mathbf{x}_i $ 간의 유사도

$$
\text{sim}(\mathbf{p}_u, \mathbf{x}_i) = \frac{\mathbf{p}_u \cdot \mathbf{x}_i}{\|\mathbf{p}_u\| \|\mathbf{x}_i\|}
$$

확장하여, 개별 특성 차원에서 유사도를 나타내면

$$
\text{sim}(\mathbf{p}_u, \mathbf{x}_i) = \frac{\sum_{j=1}^{n} p_{u,j} x_{i,j}}{\sqrt{\sum_{j=1}^{n} p_{u,j}^2} \times \sqrt{\sum_{j=1}^{n} x_{i,j}^2}}
$$

#### 4. 평점 예측
사용자 $ u $ 가 아이템 $ i $ 를 선호할 가능성을 예측

$$
\hat{r}_{u, i} = \text{sim}(\mathbf{p}_u, \mathbf{x}_i)
$$

#### 5. 추천
사용자에게 추천할 아이템 집합

$$
\hat{I}_u = \arg\max_{i \in I_{\text{all}}} \hat{r}_{u,i}
$$

여기서 $ I_{\text{all}} $ 은 전체 아이템 집합


---



## 1. Load Libraries

In [4]:
import pandas as pd
import numpy as np
import gensim
import requests
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models import Word2Vec
import json

## 2. Load Data

In [96]:
# load data
filename = "data/movies.csv"
movies = pd.read_csv(filename)

movies = movies[['movieId', 'title', 'genres']]
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [99]:
# features 만드는 방식 : 단순 concat
movies['features'] = movies['genres'] # + " " + movies['keywords'] + " " + movies['overview']
movies['features'] = movies['features'].apply(lambda x: str(x).split("|")).tolist()
print(movies['features'][0])

['Adventure', 'Animation', 'Children', 'Comedy', 'Fantasy']


## 3. Embedding Model

In [105]:
# Word2Vec 모델 학습 (skip-gram : 중심 단어로 주변 단어 예측, cbow)
# 예를 들어 "Action"이라는 단어가 "Adventure", "Fantasy", "Science"와 자주 등장하면, 이 단어들의 벡터가 비슷해지도록 학습
# pretrained도 있음. 이 모델을 가져와서 추가 학습(Fine-tuniing) 하는 것도 방법
w2v_model = Word2Vec(sentences=movies['features'], vector_size=100, window=5, min_count=2, workers=4)

# 특정 단어와 유사한 단어 확인
print(w2v_model.wv.most_similar('Animation'))

[('Adventure', 0.9980881810188293), ('Fantasy', 0.9979772567749023), ('Children', 0.9978401064872742), ('Sci-Fi', 0.9977672696113586), ('IMAX', 0.9977343082427979), ('Drama', 0.9977260231971741), ('Comedy', 0.9975833892822266), ('Action', 0.9972994327545166), ('Musical', 0.9970248341560364), ('Thriller', 0.9965717196464539)]


## 4. Item to Vector

In [122]:
def get_movie_vector(movie):
    words = movies.loc[movies['title'] == movie, 'features'].values[0]
    words = [w for w in words if w in w2v_model.wv]
    
    if len(words) == 0:
        return np.zeros(w2v_model.vector_size)
    
    return np.mean([w2v_model.wv[w] for w in words], axis=0)  # 단어 벡터들의 평균

# 모든 영화 벡터 생성
movies['vector'] = [get_movie_vector(title) for title in movies['title']]
movies.head()

,movieId,title,genres,features,vector
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,"[Adventure, Animation, Children, Comedy, Fantasy]","[-0.117267884, 0.121999286, 0.071336634, 0.005..."
1,2,Jumanji (1995),Adventure|Children|Fantasy,"[Adventure, Children, Fantasy]","[-0.12144657, 0.12178847, 0.07168645, 0.001788..."
2,3,Grumpier Old Men (1995),Comedy|Romance,"[Comedy, Romance]","[-0.09826481, 0.0962646, 0.053531766, 0.006417..."
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,"[Comedy, Drama, Romance]","[-0.10359595, 0.10317397, 0.060375456, 0.00923..."
4,5,Father of the Bride Part II (1995),Comedy,[Comedy],"[-0.11645026, 0.115399666, 0.07079423, 0.01211..."


## 5. User Profile Vector

In [201]:
# load data
filename = "data/ratings.csv"
ratings = pd.read_csv(filename)
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [202]:
def get_user_profile(userId):
    # 유저 프로파일 벡터 : sigma(유저가 본 영화의 임베딩 벡터 * 유저가 매긴 스코어) / 스코어 합 -> 가중평균
    user_rating = ratings.loc[ratings['userId'] == userId] 

    weighted_sum_vector = user_rating['movieId'].apply(
        lambda x: movies.loc[movies['movieId'] == x, 'vector'].values[0] * user_rating.loc[user_rating['movieId'] == x, 'rating'].values[0]
        ).sum()

    # 전체 가중치 합
    total_weight = user_rating['movieId'].apply(lambda x: user_rating.loc[user_rating['movieId'] == x, 'rating'].values[0]).sum()

    # 유저 프로파일 벡터 = 유저가 본 영화와 평점의 가중평균 벡터
    weighted_avg_vector = weighted_sum_vector / total_weight
    return weighted_avg_vector

# 사용자가 좋아하는 영화 입력
userId = 147
get_user_profile(userId)

array([-0.10376314,  0.10567908,  0.06159277,  0.00729661,  0.05785294,
       -0.10798232,  0.12013659,  0.31585374, -0.20936762, -0.196207  ,
        0.04944601, -0.17739224, -0.04172094, -0.00187082,  0.059193  ,
       -0.11755054,  0.14773911,  0.07373454, -0.10786869, -0.2998511 ,
       -0.03688037, -0.06463781,  0.28910893, -0.04806918, -0.0553974 ,
        0.01052841, -0.05757205,  0.13177095, -0.07854647,  0.08935115,
        0.12744816, -0.1449202 , -0.01362554, -0.11963196, -0.06939071,
        0.12325906,  0.12464396,  0.00995451, -0.05245119, -0.01083   ,
        0.12746538, -0.08727635, -0.15336004,  0.10399095,  0.06555463,
        0.01671791, -0.03815661, -0.03650375, -0.01809993,  0.04042735,
        0.0640047 , -0.17075875, -0.00633167, -0.05171117, -0.1605869 ,
       -0.08865131,  0.05384783, -0.08815134,  0.01386784, -0.0146357 ,
       -0.03657608, -0.07532576,  0.27996188, -0.05478257, -0.1190678 ,
        0.2015212 ,  0.08341964,  0.19548722, -0.21664672, -0.02

## 6. Recommendation

In [247]:
def recommend_movies(userId, top_n=10):

    user_vector = get_user_profile(userId)

    # 사용자 벡터와 모든 영화 벡터 간 유사도 계산
    sim_scores = cosine_similarity([user_vector], movies['vector'].tolist()).flatten()

    # 사용자가 본 영화 목록 가져오기
    user_watched_movies = ratings.loc[ratings['userId'] == userId, 'movieId']

    # 상위 n개 정렬 (유사도 높은 순서대로 정렬) + 본 영화 제외
    top_indices = [
        i for i in sorted(range(len(sim_scores)), key=lambda i: sim_scores[i], reverse=True)
        if movies.iloc[i]['movieId'] not in user_watched_movies
    ][:top_n]  # 상위 top_n개 선택

    # 영화 제목, 장르, 유사도 출력
    return pd.DataFrame({
        "title": movies['title'].iloc[top_indices].values,
        "genres": movies['genres'].iloc[top_indices].values,
        "similarity": sim_scores[top_indices]
    })

recommend_movies(userId, 10)

,title,genres,similarity
0,Osmosis Jones (2001),Action|Animation|Comedy|Crime|Drama|Romance|Th...,0.999904
1,"Hunting Party, The (2007)",Action|Adventure|Comedy|Drama|Thriller,0.999901
2,"Stunt Man, The (1980)",Action|Adventure|Comedy|Drama|Romance|Thriller,0.999900
3,"Protector, The (1985)",Action|Comedy|Drama|Thriller,0.999895
4,Bronson (2009),Action|Comedy|Drama|Thriller,0.999895
5,Flashback (1990),Action|Adventure|Comedy|Crime|Drama,0.999882
6,The Great Train Robbery (1978),Action|Adventure|Comedy|Crime|Drama,0.999882
7,Love Exposure (Ai No Mukidashi) (2008),Action|Comedy|Drama|Romance,0.999881
8,Dragonheart 2: A New Beginning (2000),Action|Adventure|Comedy|Drama|Fantasy|Thriller,0.999876
9,Money Train (1995),Action|Comedy|Crime|Drama|Thriller,0.999873
